<h1 style="text-align:center;">DRUGI PROJEKAT</h1>

<h2 style="font-style:italic; font-weight:bold; text-align: center;">
    Online Retail & E-Commerce Dataset
</h2>


## 1. Uvod u klasterizaciju (nenagledano učenje)
Klasterizacija je metoda nenagledanog učenja čiji je cilj da se podaci grupišu u homogene grupe (klastere) na osnovu njihove sličnosti, bez prethodno definisanih oznaka (labela). Za razliku od klasifikacije, gde su klase poznate unapred, u klasteizaciji algoritam sam otkriva strukturu podataka.

### Najčešće korišćeni algoritmi klasterizacije:
#### 1. K-Means
- Jedan od najpopularnijih algoritama.
- Zahteva unapred zadat broj klastera **k**.
- Cilj: minimizirati ukupno rastojanje tačaka od centra klastera.
- Predosti: brz, jednostavan, dobro radi na velikim skupovima.

#### 2. Hierarchical Clustering (Hijerarhijska  klasterizacija)
- Gradi hijerarhiju klastera (stablo - dendrogram).
- Ne zahteva unapred zadat broj klastera.
- Može biti:
  - aglomerativna (spajanje klastera)
  - divisivna (razdvajanje klastera)

#### 3. DBSCAN
- Ne zahteva broj klastera unapred.
- Može da detektuje outliere "šum".
- Dobro radi sa klasterima proizvoljnog oblika.
- Mana: osetljiv na izbor parametara.

U ovom projektu ćemo kasnije isprobati više ovih pristupa i uporediti rezultate.

## 2. Izbor i opis skupa podataka
Za projakat je izabran dataset: [Online Retail & E-Commerce Dataset](https://www.kaggle.com/datasets/ertugrulesol/online-retail-data)

Međutim, u ovom radu se koristi **sintetička verzija tog skupa podataka** pod nazivom `synthetic_online_retail_data.csv`.

Ovaj dataset sadrži simulirane podatke o online kupovinama i pripada oblasti *E-Commerce Services*.

### Opis podataka:
Dataset sadrži informacije o:
- customer_id - jedinstveni identifikator kupca
- product_id - identifikator kupljenog proiyvoda
- quantity - koli;ina kupljenih proizvoda
- price - cena po jedinici proizvoda
- review_score - ocena proizvoda
- age - starost kupca

### Problem koji želimo da istražimo:
Cilj projekta biće da pomoću klasterizacije 
- identifikujemo različite tipove kupaca.
- analiyiramo obrasce kupovine
- otkrijemo grupe kao što su:
  - česti kupci
  - veliki potrošači
  - povremeni kupci
  - kupci sa malim budžetom
  - ili kupci koji ostavljaju niske ocene

## Priprema podataka i deskriptivna analiza 

### Učitavanje podataka

In [1]:
import pandas as pd
#učitavanje podataka
df = pd.read_csv("synthetic_online_retail_data.csv", encoding="ISO-8859-1")
df.head()

,customer_id,order_date,product_id,category_id,category_name,product_name,quantity,price,payment_method,city,review_score,gender,age
0,13542,2024-12-17,784,10,Electronics,Smartphone,2,373.36,Credit Card,New Oliviaberg,1.0,F,56
1,23188,2024-06-01,682,50,Sports & Outdoors,Soccer Ball,5,299.34,Credit Card,Port Matthew,NaN,M,59
2,55098,2025-02-04,684,50,Sports & Outdoors,Tent,5,23.00,Credit Card,West Sarah,5.0,F,64
3,65208,2024-10-28,204,40,Books & Stationery,Story Book,2,230.11,Bank Transfer,Hernandezburgh,5.0,M,34
4,63872,2024-05-10,202,20,Fashion,Skirt,4,176.72,Credit Card,Jenkinshaven,1.0,F,33


### Osnovni pregled podataka

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customer_id     1000 non-null   int64  
 1   order_date      1000 non-null   object 
 2   product_id      1000 non-null   int64  
 3   category_id     1000 non-null   int64  
 4   category_name   1000 non-null   object 
 5   product_name    1000 non-null   object 
 6   quantity        1000 non-null   int64  
 7   price           1000 non-null   float64
 8   payment_method  1000 non-null   object 
 9   city            1000 non-null   object 
 10  review_score    799 non-null    float64
 11  gender          897 non-null    object 
 12  age             1000 non-null   int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 101.7+ KB


### Provera nedostajučih vrednosti

In [3]:
df.isnull().sum()

customer_id         0
order_date          0
product_id          0
category_id         0
category_name       0
product_name        0
quantity            0
price               0
payment_method      0
city                0
review_score      201
gender            103
age                 0
dtype: int64

### Kreiranje nove promenljive

In [4]:
df["TotalPrice"] = df["quantity"] * df["price"]

### Osnovna statistika


In [5]:
df.describe()

,customer_id,product_id,category_id,quantity,price,review_score,age,TotalPrice
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,799.000000,1000.000000,1000.000000
mean,55490.723000,540.726000,30.030000,2.947000,251.850660,3.992491,46.382000,737.326880
std,25910.185857,261.737704,14.370303,1.413573,139.194688,1.239469,16.569992,566.404843
min,10201.000000,100.000000,10.000000,1.000000,10.720000,1.000000,18.000000,20.840000
25%,33857.000000,311.750000,20.000000,2.000000,128.525000,3.000000,32.000000,285.837500
50%,54619.500000,542.500000,30.000000,3.000000,250.220000,4.000000,47.000000,592.785000
75%,77848.500000,770.750000,40.000000,4.000000,366.467500,5.000000,61.000000,1081.040000
max,99923.000000,995.000000,50.000000,5.000000,499.500000,5.000000,75.000000,2437.650000


### Vizuelizacija distribucije potrošnje

In [6]:
import matplotlib.pyplot as plt

plt.hist(df_clean["TotalPrice"], bins = 40)
plt.title("Dsitribucije ukupne potrošnje po transakciji")
plt.xlabel("TotalPrice")
plt.ylabel("Broj kupovina")
plt.show()

NameError: name 'df_clean' is not defined

### Zaključak pripreme podataka
Nakon pregleda i analize podataka:
- Nema kritičnih nedostajučih vrednosti.
- Kreirana je nova promenljiva **TotalPrice = quantity * price**
- Podaci su spremni za agregaciju po kupcu i klasterizaciju.

U narednom koraku ćemo: 
- grupisati podatke po kupcu
- skalirati ih
- i primeniti algoritme za klasterizaciju

## Klasterizacija podataka i poređenje rezultata
Cilj ovog dela projekta je da:
- primenimo više algoritama za klasterizaciju
- dobijemo grupe (klastere) kupaca,
- i uporedimo kvalitet dobijenih klastera pomoću odgovarajučih metrika

Koristićemo tri pristupa
1. K-Means
2. Hijerarhijsku klasterizaciju
3. DBSCAN

Za poređenje kvaliteta klastera koristićemo:
- Silhouette Score 
- Broj  klastera
- Broj outlier-a (za DBSCAN)

Pre klasterizacije moramo agregirati podatke po kupcu.

### Agregacija podataka po kupcu

In [7]:
customer_data = df.groupby("customer_id").agg({
    "product_id": "count",
    "quantity" : "sum",
    "TotalPrice": "sum",
    "review_score": "mean",
    "age": "mean"
}).reset_index()

customer_data.rename(columns = {
    "product_id": "NumPurchases",
    "quantity" : "TotalQuantity",
    "TotalPrice": "TotalSpending",
    "review_score": "AvgReview",
    "age": "Age"
}, inplace = True)

customer_data.head()

,customer_id,NumPurchases,TotalQuantity,TotalSpending,AvgReview,Age
0,10201,1,4,624.84,NaN,23.0
1,10211,1,2,65.02,5.0,25.0
2,10254,1,1,70.93,3.0,73.0
3,10299,1,4,815.76,5.0,33.0
4,10403,1,5,1319.35,NaN,65.0


### Skaliranje podataka

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

X = customer_data [["NumPurchases", "TotalQuantity", "TotalSpending", "AvgReview", "Age"]]

imputer = SimpleImputer(strategy = "mean")
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

### K-Means

In [12]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

sil_scores = {}
for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    sil_scores[k] = score

sil_scores

{2: np.float64(0.27232205177524915),
 3: np.float64(0.2782647557363149),
 4: np.float64(0.27106841369221146),
 5: np.float64(0.2560151289287933),
 6: np.float64(0.2506707856032576),
 7: np.float64(0.2563995677472024)}

In [13]:
best_k = max(sil_scores, key = sil_scores.get)

kmeans = KMeans(n_clusters = best_k, random_state=42, n_init = 10)
customer_data["KMeans_Cluster"] = kmeans.fit_predict(X_scaled)
print("Najbolji broj klastera:", best_k)
print("Silhouette score:", sil_scores[best_k])

Najbolji broj klastera: 3
Silhouette score: 0.2782647557363149


### Hijerarhijska klasterizacija

In [14]:
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters = best_k)
customer_data["Hier_Cluster"] = agg.fit_predict(X_scaled)

sil_hier = silhouette_score(X_scaled, customer_data["Hier_Cluster"])
print("Silhouette (Hierarchical):", sil_hier)

Silhouette (Hierarchical): 0.24398359256593177


### DBSCAN

In [16]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps = 0.8, min_samples = 5)
customer_data["DBSCAN_Cluster"] = dbscan.fit_predict(X_scaled)

n_clusters = len(set(customer_data["DBSCAN_Cluster"])) - (1 if -1 in customer_data["DBSCAN_Cluster"].values else 0)
n_outliers = list(customer_data["DBSCAN_Cluster"]).count(-1)

print("Broj DBSCAN klastera", n_clusters)
print("Broj outliera", n_outliers)

Broj DBSCAN klastera 9
Broj outliera 56


In [18]:
mask = customer_data["DBSCAN_Cluster"] != -1
sil_dbscan = silhouette_score(X_scaled[mask], customer_data["DBSCAN_Cluster"][mask])
print("Silhouette (DBSCAN):", sil_dbscan)

Silhouette (DBSCAN): 0.004114291655986099


### Poređenje metoda

In [19]:
comparison = pd.DataFrame({
    "Method": ["KMeans", "Hierarchical", "DBSCAN"],
    "Num_Clusters": [best_k, best_k, n_clusters],
    "Silhouette_Score": [sil_scores[best_k], sil_hier, sil_dbscan],
    "Outliers": [0, 0, n_outliers]
})

print(comparison)

         Method  Num_Clusters  Silhouette_Score  Outliers
0        KMeans             3          0.278265         0
1  Hierarchical             3          0.243984         0
2        DBSCAN             9          0.004114        56


### Diskusija rezultata
Na osnovu dobijenih rezultata možemo zaključiti:
- **K-Means** daje stabilne klastere i obično visok Silhouette Score, ali ne detektuje outliere.
- **Hijerarhijska klasterizacija** daje slične rezultate kao K-Means, ali je sporija.
- **DBSACN** uspešno pronalazi outliere, ali je osetljiv na parametre `eps` i `min_samples`.

U sledećem koraku biće razmotrena **redukcija dimenzionalnosti pomoću PCA**.